In [86]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import hvplot.pandas
from sklearn.decomposition import PCA
import re
import numpy as np



In [87]:
# load in movie data
team_data = pd.read_csv('/Users/adambachrach/Desktop/Yellow_Project/ml-25m/yellowteam_movie_data.csv')
movie_df = pd.DataFrame(team_data)
movie_df.reset_index().head(50)

,index,Unnamed: 0,movieId,title,genres,imdbId,tmdbId,number of ratings,rating
0,0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,57309,3.893708
1,1,1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,24228,3.251527
2,2,2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,11804,3.142028
3,3,3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,2523,2.853547
4,4,4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,11714,3.058434
5,5,5,6,Heat (1995),Action|Crime|Thriller,113277,949.0,24588,3.854909
6,6,6,7,Sabrina (1995),Comedy|Romance,114319,11860.0,12132,3.363666
7,7,7,8,Tom and Huck (1995),Adventure|Children,112302,45325.0,1344,3.114583
8,8,8,9,Sudden Death (1995),Action,114576,9091.0,3711,2.992051
9,9,9,10,GoldenEye (1995),Action|Adventure|Thriller,113189,710.0,28265,3.421458


In [88]:
# split title(year) into separate columns to later filter by year
title_years = movie_df['title']
split_title_years = title_years.str.split(r'\((.*)\)')
# convert the array into a series
split_title_years_df = split_title_years.apply(pd.Series)
split_title_years_df = split_title_years_df.rename(columns={0: 'title', 1: 'year', 2: 'misc'})
sty_df = split_title_years_df.year.str.extract('(^\d+)')
sty_df.head(50)


,0
0,1995
1,1995
2,1995
3,1995
4,1995
5,1995
6,1995
7,1995
8,1995
9,1995


In [89]:
# 
updated_movie_df = pd.merge(split_title_years_df, movie_df, left_index=True, right_index=True)
updated_movie_df.drop(['title_y', 'year', 'misc'], axis=1, inplace=True)
new_movie_df = pd.merge(updated_movie_df, sty_df, right_index=True, left_index=True)
new_movie_df.rename(columns={0: 'year'}, inplace=True)
new_movie_df.head(50)

,title_x,Unnamed: 0,movieId,genres,imdbId,tmdbId,number of ratings,rating,year
0,Toy Story,0,1,Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,57309,3.893708,1995
1,Jumanji,1,2,Adventure|Children|Fantasy,113497,8844.0,24228,3.251527,1995
2,Grumpier Old Men,2,3,Comedy|Romance,113228,15602.0,11804,3.142028,1995
3,Waiting to Exhale,3,4,Comedy|Drama|Romance,114885,31357.0,2523,2.853547,1995
4,Father of the Bride Part II,4,5,Comedy,113041,11862.0,11714,3.058434,1995
5,Heat,5,6,Action|Crime|Thriller,113277,949.0,24588,3.854909,1995
6,Sabrina,6,7,Comedy|Romance,114319,11860.0,12132,3.363666,1995
7,Tom and Huck,7,8,Adventure|Children,112302,45325.0,1344,3.114583,1995
8,Sudden Death,8,9,Action,114576,9091.0,3711,2.992051,1995
9,GoldenEye,9,10,Action|Adventure|Thriller,113189,710.0,28265,3.421458,1995


In [90]:
genres = []
for i in range(len(new_movie_df.genres)):
    for x in new_movie_df.genres[i].split('|'):
        if x not in genres:
            genres.append(x)

for x in genres:
    new_movie_df[x] = 0
for i in range(len(new_movie_df.genres)):
    for x in new_movie_df.genres[i].split('|'):
        new_movie_df[x][i] = 1

new_movie_df.drop(['genres'], axis=1)

/var/folders/n9/nmyqjt296k35d4yvkrgc5nbw0000gn/T/ipykernel_73930/1884296455.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movie_df[x][i] = 1


,title_x,Unnamed: 0,movieId,imdbId,tmdbId,number of ratings,rating,year,Adventure,Animation,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,Toy Story,0,1,114709,862.0,57309,3.893708,1995,1,1,...,0,0,0,0,0,0,0,0,0,0
1,Jumanji,1,2,113497,8844.0,24228,3.251527,1995,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Grumpier Old Men,2,3,113228,15602.0,11804,3.142028,1995,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Waiting to Exhale,3,4,114885,31357.0,2523,2.853547,1995,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Father of the Bride Part II,4,5,113041,11862.0,11714,3.058434,1995,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59042,We,59042,209157,6671244,499546.0,1,1.500000,2018,0,0,...,0,0,0,0,0,0,0,0,0,0
59043,Window of the Soul,59043,209159,297986,63407.0,1,3.000000,2001,0,0,...,0,0,0,0,1,0,0,0,0,0
59044,Bad Poems,59044,209163,6755366,553036.0,1,4.500000,2018,0,0,...,0,0,0,0,0,0,0,0,0,0
59045,A Girl Thing,59045,209169,249603,162892.0,1,3.000000,2001,0,0,...,0,0,0,0,0,0,0,0,0,1


In [91]:
numeric_df = new_movie_df.drop(['Unnamed: 0', 'title_x', 'imdbId', 'tmdbId', 'genres'], axis=1)
numeric_df.dropna(inplace=True)
numeric_df

,movieId,number of ratings,rating,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,57309,3.893708,1995,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,24228,3.251527,1995,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,11804,3.142028,1995,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,2523,2.853547,1995,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,11714,3.058434,1995,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59042,209157,1,1.500000,2018,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59043,209159,1,3.000000,2001,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
59044,209163,1,4.500000,2018,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
59045,209169,1,3.000000,2001,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [92]:
numeric_df['year'] = numeric_df['year'].astype(int)
numeric_df

,movieId,number of ratings,rating,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,57309,3.893708,1995,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,24228,3.251527,1995,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,11804,3.142028,1995,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,2523,2.853547,1995,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,11714,3.058434,1995,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59042,209157,1,1.500000,2018,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59043,209159,1,3.000000,2001,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
59044,209163,1,4.500000,2018,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
59045,209169,1,3.000000,2001,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [93]:
scores = pd.read_csv('/Users/adambachrach/Desktop/Yellow_Project/ml-25m/genome-scores.csv')
tags = pd.read_csv('/Users/adambachrach/Desktop/Yellow_Project/ml-25m/genome-tags.csv')

In [94]:
scores_df = pd.DataFrame(scores)
tags_df = pd.DataFrame(tags)

In [95]:
scores_df

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075
...,...,...,...
15584443,206499,1124,0.11000
15584444,206499,1125,0.04850
15584445,206499,1126,0.01325
15584446,206499,1127,0.14025


In [96]:
tags_df

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s
...,...,...
1123,1124,writing
1124,1125,wuxia
1125,1126,wwii
1126,1127,zombie


In [97]:
joined_tags = pd.merge(scores_df, tags_df, on='tagId')
joined_tags

,movieId,tagId,relevance,tag
0,1,1,0.02875,007
1,2,1,0.04125,007
2,3,1,0.04675,007
3,4,1,0.03425,007
4,5,1,0.04300,007
...,...,...,...,...
15584443,205072,1128,0.09100,zombies
15584444,205076,1128,0.02350,zombies
15584445,205383,1128,0.03925,zombies
15584446,205425,1128,0.02850,zombies


In [98]:
top_tags = joined_tags.loc[joined_tags.reset_index().groupby(['movieId'])['relevance'].idxmax()]
top_tags

,movieId,tagId,relevance,tag
14299560,1,1036,0.99925,toys
386849,2,29,0.97600,adventure
12434402,3,901,0.97525,sequel
15404843,4,1116,0.97525,women
12434404,5,901,0.96025,sequel
...,...,...,...,...
4600723,205072,333,0.81775,dumb but funny
6065220,205076,439,0.84750,girlie movie
10251469,205383,742,0.82050,original
13304806,205425,963,0.91825,stand-up comedy


In [99]:
unique_tags = top_tags['tagId'].unique()
len(unique_tags)

840

In [100]:
yellow_team_data = pd.read_csv('/Users/adambachrach/Desktop/Yellow_Project/yellowteam_movie_data.csv')
yellow_team_df = pd.DataFrame(yellow_team_data)
yellow_team_df

,Unnamed: 0.1,movieId,tagId,relevance,tag,Unnamed: 0,title,genres,imdbId,tmdbId,number of ratings,rating
0,0,1,1036,0.99925,toys,0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,57309,3.893708
1,1,2,29,0.97600,adventure,1,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,24228,3.251527
2,2,3,901,0.97525,sequel,2,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,11804,3.142028
3,3,4,1116,0.97525,women,3,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,2523,2.853547
4,4,5,901,0.96025,sequel,4,Father of the Bride Part II (1995),Comedy,113041,11862.0,11714,3.058434
...,...,...,...,...,...,...,...,...,...,...,...,...
13811,13811,205072,333,0.81775,dumb but funny,57790,Zombieland: Double Tap (2019),Action|Comedy|Horror,1560220,338967.0,58,3.310345
13812,13812,205076,439,0.84750,girlie movie,57792,Downton Abbey (2019),Drama,6398184,535544.0,53,3.216981
13813,13813,205383,742,0.82050,original,57900,El Camino: A Breaking Bad Movie (2019),Crime|Drama|Thriller,9243946,559969.0,252,3.642857
13814,13814,205425,963,0.91825,stand-up comedy,57918,Dave Chappelle: Sticks & Stones (2019),Comedy,10810424,624932.0,69,3.543478


In [101]:
genres = []
for i in range(len(yellow_team_df.genres)):
    for x in yellow_team_df.genres[i].split('|'):
        if x not in genres:
            genres.append(x)

for x in genres:
    yellow_team_df[x] = 0
for i in range(len(yellow_team_df.genres)):
    for x in yellow_team_df.genres[i].split('|'):
        yellow_team_df[x][i] = 1

yellow_team_df.drop(['genres'], axis=1)

/var/folders/n9/nmyqjt296k35d4yvkrgc5nbw0000gn/T/ipykernel_73930/1919174043.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yellow_team_df[x][i] = 1


,Unnamed: 0.1,movieId,tagId,relevance,tag,Unnamed: 0,title,imdbId,tmdbId,number of ratings,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,0,1,1036,0.99925,toys,0,Toy Story (1995),114709,862.0,57309,...,0,0,0,0,0,0,0,0,0,0
1,1,2,29,0.97600,adventure,1,Jumanji (1995),113497,8844.0,24228,...,0,0,0,0,0,0,0,0,0,0
2,2,3,901,0.97525,sequel,2,Grumpier Old Men (1995),113228,15602.0,11804,...,0,0,0,0,0,0,0,0,0,0
3,3,4,1116,0.97525,women,3,Waiting to Exhale (1995),114885,31357.0,2523,...,0,0,0,0,0,0,0,0,0,0
4,4,5,901,0.96025,sequel,4,Father of the Bride Part II (1995),113041,11862.0,11714,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13811,13811,205072,333,0.81775,dumb but funny,57790,Zombieland: Double Tap (2019),1560220,338967.0,58,...,1,0,0,0,0,0,0,0,0,0
13812,13812,205076,439,0.84750,girlie movie,57792,Downton Abbey (2019),6398184,535544.0,53,...,0,0,0,0,0,0,0,0,0,0
13813,13813,205383,742,0.82050,original,57900,El Camino: A Breaking Bad Movie (2019),9243946,559969.0,252,...,0,0,0,0,0,0,0,0,0,0
13814,13814,205425,963,0.91825,stand-up comedy,57918,Dave Chappelle: Sticks & Stones (2019),10810424,624932.0,69,...,0,0,0,0,0,0,0,0,0,0


In [102]:
# split title(year) into separate columns to later filter by year
title_years2 = yellow_team_df['title']
split_title_years2 = title_years2.str.split(r'\((.*)\)')
# convert the array into a series
split_title_years_df2 = split_title_years2.apply(pd.Series)
split_title_years_df2 = split_title_years_df2.rename(columns={0: 'title', 1: 'year', 2: 'misc'})
sty_df2 = split_title_years_df2.year.str.extract('(^\d+)')
sty_df2.head(50)

,0
0,1995
1,1995
2,1995
3,1995
4,1995
5,1995
6,1995
7,1995
8,1995
9,1995


In [103]:
sty_df2.reset_index()

,index,0
0,0,1995
1,1,1995
2,2,1995
3,3,1995
4,4,1995
...,...,...
13811,13811,2019
13812,13812,2019
13813,13813,2019
13814,13814,2019


In [104]:
yellow_team_years_df = pd.merge(sty_df2, yellow_team_df, right_index=True, left_index=True)
yellow_team_years_df.rename(columns={0: 'year'}, inplace=True)
yellow_team_years_df

,year,Unnamed: 0.1,movieId,tagId,relevance,tag,Unnamed: 0,title,genres,imdbId,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1995,0,1,1036,0.99925,toys,0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,...,0,0,0,0,0,0,0,0,0,0
1,1995,1,2,29,0.97600,adventure,1,Jumanji (1995),Adventure|Children|Fantasy,113497,...,0,0,0,0,0,0,0,0,0,0
2,1995,2,3,901,0.97525,sequel,2,Grumpier Old Men (1995),Comedy|Romance,113228,...,0,0,0,0,0,0,0,0,0,0
3,1995,3,4,1116,0.97525,women,3,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,...,0,0,0,0,0,0,0,0,0,0
4,1995,4,5,901,0.96025,sequel,4,Father of the Bride Part II (1995),Comedy,113041,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13811,2019,13811,205072,333,0.81775,dumb but funny,57790,Zombieland: Double Tap (2019),Action|Comedy|Horror,1560220,...,1,0,0,0,0,0,0,0,0,0
13812,2019,13812,205076,439,0.84750,girlie movie,57792,Downton Abbey (2019),Drama,6398184,...,0,0,0,0,0,0,0,0,0,0
13813,2019,13813,205383,742,0.82050,original,57900,El Camino: A Breaking Bad Movie (2019),Crime|Drama|Thriller,9243946,...,0,0,0,0,0,0,0,0,0,0
13814,2019,13814,205425,963,0.91825,stand-up comedy,57918,Dave Chappelle: Sticks & Stones (2019),Comedy,10810424,...,0,0,0,0,0,0,0,0,0,0


In [105]:
yellow_team_years_df.dropna(inplace=True)
yellow_team_years_df

,year,Unnamed: 0.1,movieId,tagId,relevance,tag,Unnamed: 0,title,genres,imdbId,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1995,0,1,1036,0.99925,toys,0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,...,0,0,0,0,0,0,0,0,0,0
1,1995,1,2,29,0.97600,adventure,1,Jumanji (1995),Adventure|Children|Fantasy,113497,...,0,0,0,0,0,0,0,0,0,0
2,1995,2,3,901,0.97525,sequel,2,Grumpier Old Men (1995),Comedy|Romance,113228,...,0,0,0,0,0,0,0,0,0,0
3,1995,3,4,1116,0.97525,women,3,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,...,0,0,0,0,0,0,0,0,0,0
4,1995,4,5,901,0.96025,sequel,4,Father of the Bride Part II (1995),Comedy,113041,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13811,2019,13811,205072,333,0.81775,dumb but funny,57790,Zombieland: Double Tap (2019),Action|Comedy|Horror,1560220,...,1,0,0,0,0,0,0,0,0,0
13812,2019,13812,205076,439,0.84750,girlie movie,57792,Downton Abbey (2019),Drama,6398184,...,0,0,0,0,0,0,0,0,0,0
13813,2019,13813,205383,742,0.82050,original,57900,El Camino: A Breaking Bad Movie (2019),Crime|Drama|Thriller,9243946,...,0,0,0,0,0,0,0,0,0,0
13814,2019,13814,205425,963,0.91825,stand-up comedy,57918,Dave Chappelle: Sticks & Stones (2019),Comedy,10810424,...,0,0,0,0,0,0,0,0,0,0


In [106]:
yellow_team_years_df['year'] = yellow_team_years_df['year'].astype(object).astype(int)

In [107]:
yellow_team_years_df.dtypes

year                    int64
Unnamed: 0.1            int64
movieId                 int64
tagId                   int64
relevance             float64
tag                    object
Unnamed: 0              int64
title                  object
genres                 object
imdbId                  int64
tmdbId                float64
number of ratings       int64
rating                float64
Adventure               int64
Animation               int64
Children                int64
Comedy                  int64
Fantasy                 int64
Romance                 int64
Drama                   int64
Action                  int64
Crime                   int64
Thriller                int64
Horror                  int64
Mystery                 int64
Sci-Fi                  int64
IMAX                    int64
Documentary             int64
War                     int64
Musical                 int64
Western                 int64
Film-Noir               int64
(no genres listed)      int64
dtype: obj

In [108]:
filtered_movies_df = yellow_team_years_df.loc[yellow_team_years_df['year'] >= 1930]
filtered_movies_df.columns

Index(['year', 'Unnamed: 0.1', 'movieId', 'tagId', 'relevance', 'tag',
       'Unnamed: 0', 'title', 'genres', 'imdbId', 'tmdbId',
       'number of ratings', 'rating', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller',
       'Horror', 'Mystery', 'Sci-Fi', 'IMAX', 'Documentary', 'War', 'Musical',
       'Western', 'Film-Noir', '(no genres listed)'],
      dtype='object')

In [109]:
tags = filtered_movies_df['tag']
len(tags.unique())

822

In [110]:
filtered_movies_df.to_csv('/Users/adambachrach/Desktop/Yellow_Project/filtered_movies.csv')
genres = filtered_movies_df[['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Mystery',
       'Sci-Fi', 'IMAX', 'Documentary', 'War', 'Musical', 'Western',
       'Film-Noir', '(no genres listed)']]

In [111]:
table = pd.pivot_table(filtered_movies_df, values=genres, index='title', columns='tag', fill_value=0)
table

(no genres listed)                     \
tag                                           007 (series) 18th century 1930s   
title                                                                           
"Great Performances" Cats (1998)                         0            0     0   
$5 a Day (2008)                                          0            0     0   
$9.99 (2008)                                             0            0     0   
'71 (2014)                                               0            0     0   
'Hellboy': The Seeds of Creation (2004)                  0            0     0   
...                                                    ...          ...   ...   
xXx (2002)                                               0            0     0   
xXx: Return of Xander Cage (2017)                        0            0     0   
xXx: State of the Union (2005)                           0            0     0   
¡Three Amigos! (1986)                                    0            0     0   
À nos amours (1983)                                      0            0     0   

                                                                 \
tag                                     1950s 1980s  7 70mm 80s   
title                                                             
"Great Performances" Cats (1998)            0     0  0    0   0   
$5 a Day (2008)                             0     0  0    0   0   
$9.99 (2008)                                0     0  0    0   0   
'71 (2014)                                  0     0  0    0   0   
'Hellboy': The Seeds of Creation (2004)     0     0  0    0   0   
...                                       ...   ... ..  ...  ..   
xXx (2002)                                  0     0  0    0   0   
xXx: Return of Xander Cage (2017)           0     0  0    0   0   
xXx: State of the Union (2005)              0     0  0    0   0   
¡Three Amigos! (1986)                       0     0  0    0   0   
À nos amours (1983)                         0     0  0    0   0   

                                                                  ...  \
tag                                     aardman studios abortion  ...   
title                                                             ...   
"Great Performances" Cats (1998)                      0        0  ...   
$5 a Day (2008)                                       0        0  ...   
$9.99 (2008)                                          0        0  ...   
'71 (2014)                                            0        0  ...   
'Hellboy': The Seeds of Creation (2004)               0        0  ...   
...                                                 ...      ...  ...   
xXx (2002)                                            0        0  ...   
xXx: Return of Xander Cage (2017)                     0        0  ...   
xXx: State of the Union (2005)                        0        0  ...   
¡Three Amigos! (1986)                                 0        0  ...   
À nos amours (1983)                                   0        0  ...   

                                               Western              \
tag                                     world politics world war i   
title                                                                
"Great Performances" Cats (1998)                     0           0   
$5 a Day (2008)                                      0           0   
$9.99 (2008)                                         0           0   
'71 (2014)                                           0           0   
'Hellboy': The Seeds of Creation (2004)              0           0   
...                                                ...         ...   
xXx (2002)                                           0           0   
xXx: Return of Xander Cage (2017)                    0           0   
xXx: State of the Union (2005)                       0           0   
¡Three Amigos! (1986)                                0           0   
À nos amours (1983)              

In [112]:
table_df = pd.DataFrame(table)
table_df

(no genres listed)                     \
tag                                           007 (series) 18th century 1930s   
title                                                                           
"Great Performances" Cats (1998)                         0            0     0   
$5 a Day (2008)                                          0            0     0   
$9.99 (2008)                                             0            0     0   
'71 (2014)                                               0            0     0   
'Hellboy': The Seeds of Creation (2004)                  0            0     0   
...                                                    ...          ...   ...   
xXx (2002)                                               0            0     0   
xXx: Return of Xander Cage (2017)                        0            0     0   
xXx: State of the Union (2005)                           0            0     0   
¡Three Amigos! (1986)                                    0            0     0   
À nos amours (1983)                                      0            0     0   

                                                                 \
tag                                     1950s 1980s  7 70mm 80s   
title                                                             
"Great Performances" Cats (1998)            0     0  0    0   0   
$5 a Day (2008)                             0     0  0    0   0   
$9.99 (2008)                                0     0  0    0   0   
'71 (2014)                                  0     0  0    0   0   
'Hellboy': The Seeds of Creation (2004)     0     0  0    0   0   
...                                       ...   ... ..  ...  ..   
xXx (2002)                                  0     0  0    0   0   
xXx: Return of Xander Cage (2017)           0     0  0    0   0   
xXx: State of the Union (2005)              0     0  0    0   0   
¡Three Amigos! (1986)                       0     0  0    0   0   
À nos amours (1983)                         0     0  0    0   0   

                                                                  ...  \
tag                                     aardman studios abortion  ...   
title                                                             ...   
"Great Performances" Cats (1998)                      0        0  ...   
$5 a Day (2008)                                       0        0  ...   
$9.99 (2008)                                          0        0  ...   
'71 (2014)                                            0        0  ...   
'Hellboy': The Seeds of Creation (2004)               0        0  ...   
...                                                 ...      ...  ...   
xXx (2002)                                            0        0  ...   
xXx: Return of Xander Cage (2017)                     0        0  ...   
xXx: State of the Union (2005)                        0        0  ...   
¡Three Amigos! (1986)                                 0        0  ...   
À nos amours (1983)                                   0        0  ...   

                                               Western              \
tag                                     world politics world war i   
title                                                                
"Great Performances" Cats (1998)                     0           0   
$5 a Day (2008)                                      0           0   
$9.99 (2008)                                         0           0   
'71 (2014)                                           0           0   
'Hellboy': The Seeds of Creation (2004)              0           0   
...                                                ...         ...   
xXx (2002)                                           0           0   
xXx: Return of Xander Cage (2017)                    0           0   
xXx: State of the Union (2005)                       0           0   
¡Three Amigos! (1986)                                0           0   
À nos amours (1983)              

In [149]:
table_df.to_csv('/Users/adambachrach/Desktop/Yellow_Project/pivot_table.csv')
table_df

(no genres listed)                     \
tag                                           007 (series) 18th century 1930s   
title                                                                           
"Great Performances" Cats (1998)                         0            0     0   
$5 a Day (2008)                                          0            0     0   
$9.99 (2008)                                             0            0     0   
'71 (2014)                                               0            0     0   
'Hellboy': The Seeds of Creation (2004)                  0            0     0   
...                                                    ...          ...   ...   
xXx (2002)                                               0            0     0   
xXx: Return of Xander Cage (2017)                        0            0     0   
xXx: State of the Union (2005)                           0            0     0   
¡Three Amigos! (1986)                                    0            0     0   
À nos amours (1983)                                      0            0     0   

                                                                 \
tag                                     1950s 1980s  7 70mm 80s   
title                                                             
"Great Performances" Cats (1998)            0     0  0    0   0   
$5 a Day (2008)                             0     0  0    0   0   
$9.99 (2008)                                0     0  0    0   0   
'71 (2014)                                  0     0  0    0   0   
'Hellboy': The Seeds of Creation (2004)     0     0  0    0   0   
...                                       ...   ... ..  ...  ..   
xXx (2002)                                  0     0  0    0   0   
xXx: Return of Xander Cage (2017)           0     0  0    0   0   
xXx: State of the Union (2005)              0     0  0    0   0   
¡Three Amigos! (1986)                       0     0  0    0   0   
À nos amours (1983)                         0     0  0    0   0   

                                                                  ...  \
tag                                     aardman studios abortion  ...   
title                                                             ...   
"Great Performances" Cats (1998)                      0        0  ...   
$5 a Day (2008)                                       0        0  ...   
$9.99 (2008)                                          0        0  ...   
'71 (2014)                                            0        0  ...   
'Hellboy': The Seeds of Creation (2004)               0        0  ...   
...                                                 ...      ...  ...   
xXx (2002)                                            0        0  ...   
xXx: Return of Xander Cage (2017)                     0        0  ...   
xXx: State of the Union (2005)                        0        0  ...   
¡Three Amigos! (1986)                                 0        0  ...   
À nos amours (1983)                                   0        0  ...   

                                            Western               \
tag                                     world war i world war ii   
title                                                              
"Great Performances" Cats (1998)                  0            0   
$5 a Day (2008)                                   0            0   
$9.99 (2008)                                      0            0   
'71 (2014)                                        0            0   
'Hellboy': The Seeds of Creation (2004)           0            0   
...                                             ...          ...   
xXx (2002)                                        0            0   
xXx: Return of Xander Cage (2017)                 0            0   
xXx: State of the Union (2005)                    0            0   
¡Three Amigos! (1986)                             0            0   
À nos amours (1983)                               0        

In [114]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(table_df)
    inertia.append(km.inertia_)

In [115]:
movie_data = {"k": k, "inertia": inertia}
table_data_df = pd.DataFrame(movie_data)
table_data_df.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

In [116]:
model = KMeans(n_clusters=5, random_state=0)
model

KMeans(n_clusters=5, random_state=0)

In [157]:
model.fit_predict(table_df)
table_df['class'] = model.labels_
table_df_sample = table_df.sample(n=60, random_state=0)
table_df_sample

(no genres listed)  \
tag                                                      007 (series)   
title                                                                   
Skipped Parts (2000)                                                0   
Anonymous (2011)                                                    0   
Munich (2005)                                                       0   
Others, The (2001)                                                  0   
High School Musical 3: Senior Year (2008)                           0   
Deuce Bigalow: Male Gigolo (1999)                                   0   
Schindler's List (1993)                                             0   
Malevolent (2018)                                                   0   
Satanic Rites of Dracula, The (1974)                                0   
Wayne's World (1992)                                                0   
I, Frankenstein (2014)                                              0   
Return to Oz (1985)                                                 0   
Man Who Knew Too Much, The (1956)                                   0   
Percy Jackson & the Olympians: The Lightning Th...                  0   
Mrs. Palfrey at the Claremont (2005)                                0   
Hellraiser: Hellworld (2005)                                        0   
Heist (2001)                                                        0   
Rookie, The (2002)                                                  0   
Guy Thing, A (2003)                                                 0   
Parineeta (2005)                                                    0   
Guns of Navarone, The (1961)                                        0   
Young & Beautiful (2013)                                            0   
Cinemania (2002)                                                    0   
Black Hawk Down (2001)                                              0   
Good Woman, A (2004)                                                0   
Punishment Park (1971)                                              0   
Wait Until Dark (1967)                                              0   
Grey Gardens (1975)                                                 0   
Friends with Money (2006)                                           0   
Lord of Illusions (1995)                                            0   
Father of the Bride (1950)                                          0   
Men Who Stare at Goats, The (2009)                                  0   
In Country (1989)                                                   0   
Nina Takes a Lover (1994)                                           0   
Impostor (2002)                                                     0   
The Darkness (2016)                                                 0   
Mummy, The (1959)                                                   0   
Hannibal (2001)                                                     0   
Brigadoon (1954)                                                    0   
Fail Safe (2000)                                                    0   
Bound (1996)                                                        0   
Smokey and the Bandit II (1980)                                     0   
Woman in Black, The (1989)                                          0   
Cabin Boy (1994)                                                    0   
Tipping the Velvet (2002)                                           0   
Navy Seals (1990)                                                   0   
Wedding Gift, The (1994)                                            0   
Ip Man: The Final Fight (2013)                                      0   
Pirates of the Caribbean: Dead Men Tell No Tale...                  0   
Carrington (1995)                                                   0   
Captain Blood (1935)                                                0   
Rude Awakening (1989)                                               0   
Wild America (1997)                 

In [143]:
filtered_to_merge = filtered_movies_df.drop(['Adventure', 'Animation', 'Children',
       'Comedy', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller',
       'Horror', 'Mystery', 'Sci-Fi', 'IMAX', 'Documentary', 'War', 'Musical',
       'Western', 'Film-Noir', '(no genres listed)'], axis=1)

filtered_to_merge = pd.get_dummies(filtered_to_merge, columns=['genres'])
filtered_to_merge

,year,Unnamed: 0.1,movieId,tagId,relevance,tag,Unnamed: 0,title,imdbId,tmdbId,...,genres_Sci-Fi,genres_Sci-Fi|IMAX,genres_Sci-Fi|Thriller,genres_Sci-Fi|Thriller|IMAX,genres_Sci-Fi|Western,genres_Thriller,genres_Thriller|War,genres_Thriller|Western,genres_War,genres_Western
0,1995,0,1,1036,0.99925,toys,0,Toy Story (1995),114709,862.0,...,0,0,0,0,0,0,0,0,0,0
1,1995,1,2,29,0.97600,adventure,1,Jumanji (1995),113497,8844.0,...,0,0,0,0,0,0,0,0,0,0
2,1995,2,3,901,0.97525,sequel,2,Grumpier Old Men (1995),113228,15602.0,...,0,0,0,0,0,0,0,0,0,0
3,1995,3,4,1116,0.97525,women,3,Waiting to Exhale (1995),114885,31357.0,...,0,0,0,0,0,0,0,0,0,0
4,1995,4,5,901,0.96025,sequel,4,Father of the Bride Part II (1995),113041,11862.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13811,2019,13811,205072,333,0.81775,dumb but funny,57790,Zombieland: Double Tap (2019),1560220,338967.0,...,0,0,0,0,0,0,0,0,0,0
13812,2019,13812,205076,439,0.84750,girlie movie,57792,Downton Abbey (2019),6398184,535544.0,...,0,0,0,0,0,0,0,0,0,0
13813,2019,13813,205383,742,0.82050,original,57900,El Camino: A Breaking Bad Movie (2019),9243946,559969.0,...,0,0,0,0,0,0,0,0,0,0
13814,2019,13814,205425,963,0.91825,stand-up comedy,57918,Dave Chappelle: Sticks & Stones (2019),10810424,624932.0,...,0,0,0,0,0,0,0,0,0,0


In [144]:
filtered_table_df = pd.merge(table_df, filtered_to_merge, on='title')
filtered_table_df.drop(['title', 'tag', 'imdbId', 'tmdbId', 'relevance', 'year'], axis=1, inplace=True)
filtered_table_df

/var/folders/n9/nmyqjt296k35d4yvkrgc5nbw0000gn/T/ipykernel_73930/1357039803.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  filtered_table_df = pd.merge(table_df, filtered_to_merge, on='title')


,"((no genres listed), 007 (series))","((no genres listed), 18th century)","((no genres listed), 1930s)","((no genres listed), 1950s)","((no genres listed), 1980s)","((no genres listed), 7)","((no genres listed), 70mm)","((no genres listed), 80s)","((no genres listed), aardman studios)","((no genres listed), abortion)",...,genres_Sci-Fi,genres_Sci-Fi|IMAX,genres_Sci-Fi|Thriller,genres_Sci-Fi|Thriller|IMAX,genres_Sci-Fi|Western,genres_Thriller,genres_Thriller|War,genres_Thriller|Western,genres_War,genres_Western
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11951,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11952,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11953,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11954,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [130]:
inertia = []
k = list(range(1, 11))

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(filtered_table_df)
    inertia.append(km.inertia_)

/Users/adambachrach/Desktop/Python_Venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
/Users/adambachrach/Desktop/Python_Venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
/Users/adambachrach/Desktop/Python_Venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
/Users/adambachrach/Desktop/Python_Venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names 

In [131]:
movie_data2 = {"k": k, "inertia": inertia}
filtered_table_data_df = pd.DataFrame(movie_data2)
filtered_table_data_df.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)